In [66]:
import csv
import requests
import json
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
def get_one_page(url):
    try:
        response = requests.get(url,headers = headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None
text=json.loads(get_one_page("https://movie.douban.com/j/search_subjects?type=movie&tag=%E7%83%AD%E9%97%A8&sort=recommend&page_limit=4000"))['subjects']

In [69]:
#python2可以用file替代open
with open("test.csv","w",newline='') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(text[0].keys())
    for i in text:
        writer.writerow(i.values())
    #写入多行用writerows

In [67]:
#首先，安装docx 包，pip install python-docx
from docx import Document
from docx.shared import Pt
import re
from bs4 import BeautifulSoup
import urllib
document = Document('201692117 张金哲 python第三次作业，截图部分.docx')
for i in text:
    urllib.request.urlretrieve(i['cover'],'1.jpg')
    p=document.add_paragraph().add_run(i['title'])
    p.font.bold=True
    p.font.size=Pt(20)
    p = document.add_paragraph('得分：')
    p.add_run(i['rate']).italic=True
    p.add_run("分")
    p = document.add_paragraph('详情请见：')
    p.add_run(i['url']).italic=True
    document.add_picture('1.jpg', width=Pt(150)) #插入图片
    a=BeautifulSoup(get_one_page(i['url']),'html.parser')
    document.add_paragraph(a.find('div',id=re.compile('info')).text+"电影评价：\n"+a.find('div',id=re.compile('link-report')).text.replace("                                ", "")+"\n\n\n")

# document.add_heading('Heading, level 1', level=1)
# document.add_paragraph('Intense quote', style='IntenseQuote')

# document.add_paragraph(
#     'first item in unordered list', style='ListBullet'
# )
# document.add_paragraph(
#     'first item in ordered list', style='ListNumber'
# )
table = document.add_table(rows=len(text)+1, cols=len(text[0])) #插入表格
for ii,i in enumerate(text[0].keys()):
    table.rows[0].cells[ii].text=i
for ii,i in enumerate(text):
    for jj,j in enumerate(i.values()):
        table.rows[ii+1].cells[jj].text=str(j)

document.add_page_break()

document.save('201692117 张金哲 python第三次作业.docx')  #保存文档